In [83]:
pip install gigachain_community

Note: you may need to restart the kernel to use updated packages.


In [84]:
from langchain_community.chat_models.gigachat import GigaChat

In [85]:
credentials="NmZjNjg0NjktZDI2ZS00M2FmLTk0NmMtMTdmODljNjM4ZWZmOjBiZmQ1ZDI2LTczODUtNDQxYi04OWQyLWJlNTUzN2QzZWI4NQ=="

In [86]:
def load_user_prompt(file_path): # функция загрузки user_prompt из txt файла
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read().strip()

In [ ]:
SYSTEM_PROMPT = """
ВЫВОД ДОЛЖЕН НАЧИНАТЬСЯ СТРОГО С {"scenes": [ И НИЧЕГО ПЕРЕД ЭТИМ!
Роль: Профессиональный геймдизайнер RPG, специализирующийся на ветвящихся сюжетах и атмосферных мирах.

Требования к генерации:
1. Формат:
- Строго валидный JSON
- Только объект с массивом "scenes"
- Никаких посторонних символов/комментариев

2. Входные данные (заменять фигурными скобками):
- Жанр: {жанр}
- Герой: {герой} 
- Цель: {цель}

3. Структура квеста:
- 5-10 сцен
- Минимум 1 развилка с альтернативной веткой (3+ сцены)
- Все сцены кроме финальных: 2 или 3 выбора
- Финальные сцены: choices = []

4. Технические требования:
- Все scene_id: lowercase_underscore
- Длина text в сцене: 100-500 символов
- Длина text в выборе: 20-100 символов
- Все next_scene должны существовать
- Недопустимы циклы в сценах

5. Качественные критерии:
- Яркая передача атмосферы жанра
- Логичные последствия выборов
- Минимум 1 неочевидный выбор
- Реалистичные NPC с динамичными реакциями
- Соответствие структуре: завязка->конфликт->развязка

Процесс генерации:
1. Создать outline с scene_id
2. Проверить ветвление
3. Наполнить контентом
4. Валидировать JSON
5. Убедиться в выполнении всех требований

Запрещено:
- Нарушать формат JSON
- Создавать невалидные ссылки между сценами
- Использовать неподходящий контент
- Добавлять пояснения к выводу

Шаблон для заполнения:
{
  "scenes": [
    {
      "scene_id": "start",
      "text": "Вы - {герой} в мире {жанр}. {Атмосферное описание}. Ваша цель: {цель}.",
      "choices": [
        {
          "text": "{Логичный выбор 1}",
          "next_scene": "scene_1"
        },
        {
          "text": "{Альтернативный выбор 2}",
          "next_scene": "scene_2"
        }
      ]
    }
    // Дополнить 4-9 сценами
  ]
}

Жесткие технические требования:
1. Вывод ТОЛЬКО в формате JSON без каких-либо посторонних символов
2. Начинаться должен строго с: {"scenes": [
3. Заканчиваться строго на: ]}
4. Никаких комментариев, пояснений или оберток

ВАЖНО: вывод должен быть в чистом JSON, без пояснений или комментариев. Поле scenes обязательно должно быть
"""

In [88]:
from httpx import ReadTimeout

In [89]:
# функция для генерации квеста
# Загружаем user_prompt
def generate_rpg_quest(user_prompt_txt_path, credentials):
    user_prompt = load_user_prompt(user_prompt_txt_path)

    try:
        giga = GigaChat(
            credentials=credentials,
            verify_ssl_certs=False,
            timeout=360,
            model="GigaChat-2-Max"
        )

        full_prompt = f"{SYSTEM_PROMPT}\n\nВходные данные:\n{user_prompt}\n\nВывод только в JSON!"

        response = giga.invoke(full_prompt)

        response_text = response.content

        return response_text

    except ReadTimeout:
        print("Ошибка: превышено время ожидания ответа от GigaChat.")
        return {"error": "timeout"}
    except Exception as e:
        print(f"Критическая ошибка: {str(e)}")
        return {"error": str(e)}


In [90]:
import json

In [91]:
if __name__ == "__main__":
    # Путь к файлу с user_prompt (например, "input.txt")
    USER_PROMPT_PATH = "input/example-3.txt"  # Замените на ваш путь

    # Авторизационный ключ GigaChat
    CREDENTIALS = credentials  # Замените на реальный ключ

    # Генерация квеста
    quest = generate_rpg_quest(USER_PROMPT_PATH, CREDENTIALS)

    # Сохранение результата в файл
    with open("text_output/example-3.txt", "w", encoding="utf-8") as f:
        f.write(quest)

    print("Квест успешно сгенерирован и сохранён в text_output/example-3.txt!")

Квест успешно сгенерирован и сохранён в text_output/example-3.txt!


In [94]:
import subprocess
import os

def process_quest_with_validation(quest_name):
    """
    Обрабатывает квест через process.py, который:
    1. Читает text_output/{quest_name}.txt
    2. Конвертирует в JSON
    3. Сохраняет в generated_quests/{quest_name}.json
    4. Выполняет валидацию
    """
    try:
        # Проверяем существование входного файла
        input_file = f"text_output/{quest_name}.txt"
        if not os.path.exists(input_file):
            print(f"Ошибка: файл {input_file} не найден")
            return False
        
        print(f"Обрабатываем квест: {quest_name}")
        print(f"Входной файл: {input_file}")
        
        # Запускаем process.py
        result = subprocess.run([
            'python3', 'process.py', f"text_output/{quest_name}.txt", "-o", f"generated_quests/{quest_name}.json"
        ], 
        capture_output=True, 
        text=True
        )
        
        # Выводим результат
        if result.stdout:
            print("Вывод process.py:")
            print(result.stdout)
        
        if result.stderr:
            print("Ошибки:")
            print(result.stderr)
        
        if result.returncode == 0:
            print(f"✅ Квест {quest_name} успешно обработан и валидирован!")
            
            # Проверяем, что файл создался
            output_file = f"generated_quests/{quest_name}.json"
            if os.path.exists(output_file):
                print(f"✅ Файл {output_file} создан")
            else:
                print(f"⚠️ Файл {output_file} не найден")
            
            return True
        else:
            print(f"❌ Ошибка при обработке квеста {quest_name}")
            return False
            
    except Exception as e:
        print(f"Критическая ошибка: {e}")
        return False

# Обрабатываем example-3
if __name__ == "__main__":
    quest_name = "invalid"
    
    print(f"Запускаем обработку квеста: {quest_name}")
    print("=" * 50)
    
    success = process_quest_with_validation(quest_name)
    
    if success:
        print("\n🎉 Обработка завершена успешно!")
        print(f"Квест {quest_name} готов к использованию в generated_quests/{quest_name}.json")
    else:
        print("\n❌ Обработка завершилась с ошибками")
        print("Проверьте логи выше для диагностики проблем")

Запускаем обработку квеста: invalid
Обрабатываем квест: invalid
Входной файл: text_output/invalid.txt
Вывод process.py:
Результат валидации: Файл не является валидным JSON. Ошибка: Expecting value: line 1 column 1 (char 0)

❌ Ошибка при обработке квеста invalid

❌ Обработка завершилась с ошибками
Проверьте логи выше для диагностики проблем
